In [8]:
# the OG
import cv2
import numpy as np
import sys
video_path =r"D:\a netfix\ML b&w colorizer Project\5614724-hd_720_1280_30fps.mp4"
print(video_path)
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
  print('NOT OPENED')
proto = r'D:\a netfix\ML b&w colorizer Project\models\colorization_deploy_v2.prototxt'
weights = r'D:\a netfix\ML b&w colorizer Project\models\colorization_release_v2.caffemodel'
pts_in_hull_path = r'D:\a netfix\ML b&w colorizer Project\models\pts_in_hull.npy'

# Load cluster centers
pts_in_hull = np.load(pts_in_hull_path)
pts_in_hull = pts_in_hull.transpose().reshape(2, 313, 1, 1).astype(np.float32)

# Load model
net = cv2.dnn.readNetFromCaffe(proto, weights)

# Populate cluster centers as 1x1 convolution kernel
net.getLayer(net.getLayerId('class8_ab')).blobs = [pts_in_hull]
net.getLayer(net.getLayerId('conv8_313_rh')).blobs = [np.full((1, 313), 2.606, np.float32)]

skipping = False

# Initialize video output
output_size = (
    640,
    int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) * 640 / cap.get(cv2.CAP_PROP_FRAME_WIDTH))
)
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
out = cv2.VideoWriter('%s_output.mp4' % (video_path.split('.')[0]), fourcc, cap.get(cv2.CAP_PROP_FPS), output_size)

n_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)

while True:
    ret, img = cap.read()

    if not ret:
        break

    img = cv2.resize(img, output_size)

    pred_bgr = img.copy()
    img_ori = img.copy()

    # Normalize input
    img_ori = cv2.cvtColor(img_ori, cv2.COLOR_BGR2GRAY)
    img_ori = cv2.cvtColor(img_ori, cv2.COLOR_GRAY2RGB)

    if not skipping:
        img_ori = (img_ori / 255.).astype(np.float32)

        # Convert RGB to LAB
        img_lab = cv2.cvtColor(img_ori, cv2.COLOR_RGB2Lab)
        img_l = img_lab[:, :, 0]

        input_img = cv2.resize(img_l, (224, 224))
        input_img -= 50  # Subtract 50 for mean-centering

        # Prediction
        net.setInput(cv2.dnn.blobFromImage(input_img))
        pred = net.forward()[0, :, :, :].transpose((1, 2, 0))

        # Resize to original image shape
        pred_resize = cv2.resize(pred, (img.shape[1], img.shape[0]))

        # Concatenate with original image L
        pred_lab = np.concatenate([img_l[:, :, np.newaxis], pred_resize], axis=2)

        # Convert LAB to RGB
        pred_bgr = cv2.cvtColor(pred_lab, cv2.COLOR_Lab2BGR)
        pred_bgr = np.clip(pred_bgr, 0, 1) * 255
        pred_bgr = pred_bgr.astype(np.uint8)

    # Write the output frame to video
    out.write(pred_bgr)

    key = cv2.waitKey(1)
    if key == ord('q'):
        break
    elif key == ord('s'):
        skipping = not skipping

    print('%s/%s' % (cap.get(cv2.CAP_PROP_POS_FRAMES), n_frames), end='\r')

cap.release()
out.release()
cv2.destroyAllWindows()


In [1]:
import cv2
import numpy as np
import ipywidgets as widgets
from IPython.display import display, Image, clear_output
import time
import os
import threading
import subprocess
import shutil

# Paths
FFMPEG_PATH = r"C:\ffmpeg\ffmpeg.exe"
FFPROBE_PATH = r"C:\ffmpeg\ffprobe.exe"
output_dir = r"D:\a netfix\ML b&w colorizer Project"

# Model files
proto = os.path.join(output_dir, "models", "colorization_deploy_v2.prototxt")
weights = os.path.join(output_dir, "models", "colorization_release_v2.caffemodel")
pts_in_hull_path = os.path.join(output_dir, "models", "pts_in_hull.npy")

# Load model and cluster centers
net = cv2.dnn.readNetFromCaffe(proto, weights)
pts_in_hull = np.load(pts_in_hull_path).transpose().reshape(2, 313, 1, 1).astype(np.float32)
net.getLayer(net.getLayerId("class8_ab")).blobs = [pts_in_hull]
net.getLayer(net.getLayerId("conv8_313_rh")).blobs = [np.full((1, 313), 2.606, np.float32)]

# UI widgets
file_input = widgets.Text(placeholder='Enter full path to image or video...', layout=widgets.Layout(width='100%'))
colorize_btn = widgets.Button(description="🎨 Colorize", button_style='success')
bw_btn = widgets.Button(description="⚫ Convert to B&W", button_style='danger')
status_output = widgets.Output()
timer_label = widgets.Label(value="⏱️ Timer: 0.0 seconds")
frame_label = widgets.Label(value="🧮 Frame: 0/0")

def update_timer(start_time, stop_flag):
    while not stop_flag[0]:
        elapsed = time.time() - start_time[0]
        timer_label.value = f"⏱️ Timer: {elapsed:.1f} seconds"
        time.sleep(0.1)

def has_audio(video_path):
    try:
        result = subprocess.run([
            FFPROBE_PATH, "-v", "error", "-select_streams", "a", "-show_entries",
            "stream=codec_type", "-of", "csv=p=0", video_path
        ], capture_output=True, text=True)
        return "audio" in result.stdout
    except:
        return False

def preview_media(filepath):
    ext = os.path.splitext(filepath)[-1].lower()
    if ext in ['.jpg', '.jpeg', '.png']:
        display(Image(filename=filepath))
    # Video preview intentionally removed

def colorize_frame(frame):
    frame = cv2.resize(frame, (640, int(frame.shape[0] * 640 / frame.shape[1])))
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray_rgb = cv2.cvtColor(gray, cv2.COLOR_GRAY2RGB)
    img_input = (gray_rgb / 255.).astype(np.float32)
    img_lab = cv2.cvtColor(img_input, cv2.COLOR_RGB2Lab)
    img_l = img_lab[:, :, 0]
    input_img = cv2.resize(img_l, (224, 224)) - 50
    net.setInput(cv2.dnn.blobFromImage(input_img))
    pred = net.forward()[0].transpose((1, 2, 0))
    pred_resize = cv2.resize(pred, (frame.shape[1], frame.shape[0]))
    pred_lab = np.concatenate([img_l[:, :, np.newaxis], pred_resize], axis=2)
    pred_bgr = cv2.cvtColor(pred_lab, cv2.COLOR_Lab2BGR)
    return np.clip(pred_bgr, 0, 1) * 255

def convert_to_bw(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    return cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)

def process_media(file_path, mode="colorize"):
    with status_output:
        clear_output()
        try:
            start, stop = [time.time()], [False]
            threading.Thread(target=update_timer, args=(start, stop)).start()

            if not os.path.exists(file_path):
                raise FileNotFoundError("❌ File does not exist!")

            ext = os.path.splitext(file_path)[-1].lower()
            output_path = os.path.join(output_dir, f"{mode}_{os.path.basename(file_path)}")

            if ext in ['.jpg', '.jpeg', '.png']:
                img = cv2.imread(file_path)
                if mode == "colorize":
                    out_img = colorize_frame(img)
                else:
                    out_img = convert_to_bw(img)
                cv2.imwrite(output_path, out_img.astype(np.uint8))
                stop[0] = True
                preview_media(output_path)
                print(f"✅ Output saved at: {output_path}")

            elif ext in ['.mp4', '.avi', '.mov', '.mkv']:
                cap = cv2.VideoCapture(file_path)
                if not cap.isOpened():
                    raise ValueError("❌ Failed to open video")

                fps = cap.get(cv2.CAP_PROP_FPS)
                total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                width = 640
                height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) * width / cap.get(cv2.CAP_PROP_FRAME_WIDTH))

                temp_video = os.path.join(output_dir, "temp_no_audio.mp4")
                fourcc = cv2.VideoWriter_fourcc(*'mp4v')
                out = cv2.VideoWriter(temp_video, fourcc, fps, (width, height))

                frame_idx = 0
                while True:
                    ret, frame = cap.read()
                    if not ret:
                        break
                    frame = cv2.resize(frame, (width, height))
                    if mode == "colorize":
                        frame_out = colorize_frame(frame)
                    else:
                        frame_out = convert_to_bw(frame)
                    out.write(frame_out.astype(np.uint8))
                    frame_idx += 1
                    frame_label.value = f"🧮 Frame: {frame_idx}/{total_frames}"

                cap.release()
                out.release()

                if has_audio(file_path):
                    subprocess.run([
                        FFMPEG_PATH,
                        "-y", "-i", temp_video,
                        "-i", file_path,
                        "-c:v", "copy", "-c:a", "aac",
                        "-map", "0:v:0", "-map", "1:a:0?",
                        output_path
                    ])
                    os.remove(temp_video)
                else:
                    shutil.move(temp_video, output_path)

                stop[0] = True
                # No video preview here
                print(f"✅ Output saved at: {output_path}")
            else:
                raise ValueError("❌ Unsupported file type.")
        except Exception as e:
            stop[0] = True
            print(f"❌ Error: {e}")

def handle_colorize(b):
    clear_output(wait=True)
    display(file_input, colorize_btn, bw_btn, timer_label, frame_label, status_output)
    process_media(file_input.value.strip('"'), mode="colorize")

def handle_bw(b):
    clear_output(wait=True)
    display(file_input, colorize_btn, bw_btn, timer_label, frame_label, status_output)
    process_media(file_input.value.strip('"'), mode="bw")

colorize_btn.on_click(handle_colorize)
bw_btn.on_click(handle_bw)

# Display UI
display(file_input, colorize_btn, bw_btn, timer_label, frame_label, status_output)


error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\caffe\caffe_io.cpp:1126: error: (-2:Unspecified error) FAILED: fs.is_open(). Can't open "D:\a netfix\ML b&w colorizer Project\models\colorization_deploy_v2.prototxt" in function 'cv::dnn::ReadProtoFromTextFile'
